# Cross-correlation among cables

In [1]:
from pathlib import Path
import os

os.chdir(Path(os.getcwd()).parent)

In [2]:
from AnomalyCableDetection.load import Loader, Preprocessor
from AnomalyCableDetection.stl import CableSTL, CrossCorrelation, AdjacencyType
from AnomalyCableDetection.plot import *

In [3]:
from os.path import join
from pathlib import Path

import matplotlib.dates as mdates
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
import glob
import os
import re

In [4]:
cc_path = join(Path(os.getcwd()), 'results', 'cross_correlation', 'raw_period_3600')
residual_path = join(cc_path, 'RESIDUAL', 'hourly')

In [5]:
cc = CrossCorrelation('raw_period_3600', '2009-05-05', True)
stl_dict = cc.stl_dict
date_list = cc.date_list
cc.cable_list.remove('SJX08')
cc.cable_list.remove('SJS13')
cc.cable_list.remove('SJX13')
cable_list = cc.cable_list
cable_list

['SJS08',
 'SJS09',
 'SJS10',
 'SJS11',
 'SJS12',
 'SJS14',
 'SJX09',
 'SJX10',
 'SJX11',
 'SJX12',
 'SJX14']

In [7]:
def get_cross_correlation_among_cables(stl_dict, date, cable_names, stl_type, cc_path):
    cc_df = pd.DataFrame(index=cable_names, columns=[i for i in range(24)])

    for i in range(24):
        for cable1 in cable_names:
            ad_cables = get_adjacency_cable_list(cable1)
            print(cable1, ad_cables)
            cc_list = []
            for cable2 in ad_cables:
                if cable1 == cable2: continue
                value1 = stl_dict[date, cable1, i].loc[:, stl_type]
                value2 = stl_dict[date, cable2, i].loc[:, stl_type]
                cross_correlation = np.correlate(value1, value2)[0]
                cc_list.append(cross_correlation)
            print(len(cc_list))
            if len(cc_list) == 0:
                mean_cc = np.nan
            else:
                mean_cc = sum(cc_list) / float(len(cc_list))
                cc_df.loc[cable1, i] = mean_cc
    ccc_path = join(cc_path, f'{date}.csv')
    cc_df.to_csv(ccc_path)
    print(f'save cross correlation date in {date} to {ccc_path}')
    return cc_df

def get_adjacency_cable_list(cable_name):
    cable_list = [cable_name]
    cable_list.extend(get_across_cable(cable_name))
    cable_list.extend(get_side_cables(cable_name))

    if 'SJX08' in cable_list:
        cable_list.remove('SJX08')
    elif 'SJS13' in cable_list:
        cable_list.remove('SJS13')
    elif 'SJX13' in cable_list:
        cable_list.remove('SJX13')
        
    return cable_list

def get_across_cable(cable_name):
    if cable_name[2] == 'S':
        return [f'SJX{cable_name[-2:]}']
    else:
        return [f'SJS{cable_name[-2:]}']

def get_side_cables(cable_name):
    number = int(cable_name[-2:])
    cable_names = []
    if number != 8:
        cable_names.append(f'{cable_name[:3]}{str(number - 1).zfill(2)}')
    elif number != 14:
        cable_names.append(f'{cable_name[:3]}{str(number + 1).zfill(2)}')

    return cable_names

original_dict = dict()
date_list = ['2009-05-05']
for date in date_list:
    cc_df = get_cross_correlation_among_cables(stl_dict, date, cable_list, 'residual', residual_path)
    original_dict.update({date: cc_df})

SJS08 ['SJS08', 'SJS09']
1
SJS09 ['SJS09', 'SJX09', 'SJS08']
2
SJS10 ['SJS10', 'SJX10', 'SJS09']
2
SJS11 ['SJS11', 'SJX11', 'SJS10']
2
SJS12 ['SJS12', 'SJX12', 'SJS11']
2
SJS14 ['SJS14', 'SJX14']
1
SJX09 ['SJX09', 'SJS09']
1
SJX10 ['SJX10', 'SJS10', 'SJX09']
2
SJX11 ['SJX11', 'SJS11', 'SJX10']
2
SJX12 ['SJX12', 'SJS12', 'SJX11']
2
SJX14 ['SJX14', 'SJS14']
1
SJS08 ['SJS08', 'SJS09']
1
SJS09 ['SJS09', 'SJX09', 'SJS08']
2
SJS10 ['SJS10', 'SJX10', 'SJS09']
2
SJS11 ['SJS11', 'SJX11', 'SJS10']
2
SJS12 ['SJS12', 'SJX12', 'SJS11']
2
SJS14 ['SJS14', 'SJX14']
1
SJX09 ['SJX09', 'SJS09']
1
SJX10 ['SJX10', 'SJS10', 'SJX09']
2
SJX11 ['SJX11', 'SJS11', 'SJX10']
2
SJX12 ['SJX12', 'SJS12', 'SJX11']
2
SJX14 ['SJX14', 'SJS14']
1
SJS08 ['SJS08', 'SJS09']
1
SJS09 ['SJS09', 'SJX09', 'SJS08']
2
SJS10 ['SJS10', 'SJX10', 'SJS09']
2
SJS11 ['SJS11', 'SJX11', 'SJS10']
2
SJS12 ['SJS12', 'SJX12', 'SJS11']
2
SJS14 ['SJS14', 'SJX14']
1
SJX09 ['SJX09', 'SJS09']
1
SJX10 ['SJX10', 'SJS10', 'SJX09']
2
SJX11 ['SJX11', 'SJ